# Database Admin 101 - Lab

## Introduction 

In this lab, you'll go through the process of designing and creating a database. From there, you'll begin to populate this table with mock data provided to you.

## Objectives

You will be able to:

* Use knowledge of the structure of databases to create a database and populate it

## The Scenario

You are looking to design a database for a school that will house various information from student grades to contact information, class roster lists and attendance. First, think of how you would design such a database. What tables would you include? What columns would each table have? What would be the primary means to join said tables?

## Creating the Database

Now that you've put a little thought into how you might design your database, it's time to go ahead and create it! Start by import the necessary packages. Then, create a database called **school.sqlite**.

In [1]:
# Import necessary packages
import sqlite3
import pandas as pd

In [2]:
# Create the database school.sqlite 
conn = sqlite3.connect('school.sqlite')
cursor = conn.cursor()

## Create a Table for Contact Information

Create a table called contactInfo to house contact information for both students and staff. Be sure to include columns for first name, last name, role (student/staff), telephone number, street, city, state, and zipcode. Be sure to also create a primary key for the table. 

In [3]:
conn.execute('''
DROP TABLE contactinfo;
''')

In [4]:
# Your code here
command1 = '''
CREATE TABLE contactinfo (
       contact_id INTEGER PRIMARY KEY AUTOINCREMENT,
       firstname VARCHAR(50),
       lastname VARCHAR(60),
       role TEXT,
       telephone_number VARCHAR(13),
       street VARCHAR(100),
       city VARCHAR(60),
       state VARCHAR(50),
       zipcode VARCHAR(10)
)
'''
cursor.execute(command1)

In [5]:
# Load the list of dictionaries; just run this cell
import pickle

with open('contact_list.pickle', 'rb') as f:
    contacts = pickle.load(f)

In [6]:
contacts

[{'firstName': 'Christine',
  'lastName': 'Holden',
  'role': 'staff',
  'telephone ': 2035687697,
  'street': '1672 Whitman Court',
  'city': 'Stamford',
  'state': 'CT',
  'zipcode ': '06995'},
 {'firstName': 'Christopher',
  'lastName': 'Warren',
  'role': 'student',
  'telephone ': 2175150957,
  'street': '1935 University Hill Road',
  'city': 'Champaign',
  'state': 'IL',
  'zipcode ': '61938'},
 {'firstName': 'Linda',
  'lastName': 'Jacobson',
  'role': 'staff',
  'telephone ': 4049446441,
  'street': '479 Musgrave Street',
  'city': 'Atlanta',
  'state': 'GA',
  'zipcode ': '30303'},
 {'firstName': 'Andrew',
  'lastName': 'Stepp',
  'role': 'student',
  'telephone ': 7866419252,
  'street': '2981 Lamberts Branch Road',
  'city': 'Hialeah',
  'state': 'Fl',
  'zipcode ': '33012'},
 {'firstName': 'Jane',
  'lastName': 'Evans',
  'role': 'student',
  'telephone ': 3259909290,
  'street': '1461 Briarhill Lane',
  'city': 'Abilene',
  'state': 'TX',
  'zipcode ': '79602'},
 {'firstNa

In [7]:
# Iterate over the contact list and populate the contactInfo table here
contact_info = []

for contact in contacts:
    contact_info.append(
        (
        contact.get('firstName'),
        contact.get('lastName'),
        contact.get('role'),
        contact.get('telephone '),
        contact.get('street'),
        contact.get('city'),
        contact.get('state'),
        contact.get('zipcode ')
        )
    )

command2 = '''
INSERT INTO contactinfo(firstname, lastname, role, telephone_number, street, city, state, zipcode)
VALUES(?,?,?,?,?,?,?,?);
'''

cursor.executemany(command2, contact_info)


**Query the Table to Ensure it is populated**

In [8]:
pd.read_sql_query('''SELECT * FROM contactinfo;''', conn, index_col='contact_id')

,firstname,lastname,role,telephone_number,street,city,state,zipcode
contact_id,,,,,,,,
1,Christine,Holden,staff,2035687697,1672 Whitman Court,Stamford,CT,06995
2,Christopher,Warren,student,2175150957,1935 University Hill Road,Champaign,IL,61938
3,Linda,Jacobson,staff,4049446441,479 Musgrave Street,Atlanta,GA,30303
4,Andrew,Stepp,student,7866419252,2981 Lamberts Branch Road,Hialeah,Fl,33012
5,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602
6,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602
7,Mary,Raines,student,9075772295,3975 Jerry Toth Drive,Ninilchik,AK,99639
8,Ed,Lyman,student,5179695576,3478 Be Sreet,Lansing,MI,48933


## Commit Your Changes to the Database

Persist your changes by committing them to the database.

In [9]:
# Your code here
conn.commit()

## Create a Table for Student Grades

Create a new table in the database called "grades". In the table, include the following fields: userId, courseId, grade.

** This problem is a bit more tricky and will require a dual key. (A nuance you have yet to see.)
Here's how to do that:

```SQL
CREATE TABLE table_name(
   column_1 INTEGER NOT NULL,
   column_2 INTEGER NOT NULL,
   ...
   PRIMARY KEY(column_1,column_2,...)
);
```

In [10]:
# Create the grades table
command3 = '''
CREATE TABLE grades(
       userId INTEGER NOT NULL,
       courseId INTEGER NOT NULL,
       grade TEXT,
       PRIMARY KEY(userId, courseId)
)
'''

cursor.execute(command3)

In [11]:
pd.read_sql('''SELECT * FROM grades''', conn)

,userId,courseId,grade


## Remove Duplicate Entries

An analyst just realized that there is a duplicate entry in the contactInfo table! Find and remove it.

In [12]:
# Find the duplicate entry
command4 = '''
SELECT * FROM contactinfo
GROUP BY firstname, lastname, role, telephone_number, street, city, state, zipcode
HAVING COUNT(*) > 1;
'''

pd.read_sql(command4, conn)


,contact_id,firstname,lastname,role,telephone_number,street,city,state,zipcode
0,5,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602


In [14]:
# Delete the duplicate entry
command5 = '''
DELETE FROM contactinfo WHERE contact_id = 5;
'''

cursor.execute(command5)

In [15]:
# Check that the duplicate entry was removed
command6 = '''
SELECT * FROM contactinfo
GROUP BY firstname, lastname, role, telephone_number, street, city, state, zipcode
HAVING COUNT(*) > 1;
'''

pd.read_sql(command6, conn)

,contact_id,firstname,lastname,role,telephone_number,street,city,state,zipcode


In [16]:
pd.read_sql_query('''SELECT * FROM contactinfo;''', conn, index_col='contact_id')

,firstname,lastname,role,telephone_number,street,city,state,zipcode
contact_id,,,,,,,,
1,Christine,Holden,staff,2035687697,1672 Whitman Court,Stamford,CT,06995
2,Christopher,Warren,student,2175150957,1935 University Hill Road,Champaign,IL,61938
3,Linda,Jacobson,staff,4049446441,479 Musgrave Street,Atlanta,GA,30303
4,Andrew,Stepp,student,7866419252,2981 Lamberts Branch Road,Hialeah,Fl,33012
6,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602
7,Mary,Raines,student,9075772295,3975 Jerry Toth Drive,Ninilchik,AK,99639
8,Ed,Lyman,student,5179695576,3478 Be Sreet,Lansing,MI,48933


## Updating an Address

Ed Lyman just moved to `2910 Simpson Avenue York, PA 17403`. Update his address accordingly.

In [17]:
# Update Ed's address
command7 = '''
UPDATE contactinfo 
SET street = '2910 Simpson Avenue',
    city = 'York',
    state = 'PA',
    zipcode = '17403'
WHERE firstname = 'Ed';
'''

cursor.execute(command7)


In [18]:
# Query the database to ensure the change was made
pd.read_sql_query('''SELECT * FROM contactinfo;''', conn, index_col='contact_id')

,firstname,lastname,role,telephone_number,street,city,state,zipcode
contact_id,,,,,,,,
1,Christine,Holden,staff,2035687697,1672 Whitman Court,Stamford,CT,06995
2,Christopher,Warren,student,2175150957,1935 University Hill Road,Champaign,IL,61938
3,Linda,Jacobson,staff,4049446441,479 Musgrave Street,Atlanta,GA,30303
4,Andrew,Stepp,student,7866419252,2981 Lamberts Branch Road,Hialeah,Fl,33012
6,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602
7,Mary,Raines,student,9075772295,3975 Jerry Toth Drive,Ninilchik,AK,99639
8,Ed,Lyman,student,5179695576,2910 Simpson Avenue,York,PA,17403


## Commit Your Changes to the Database

Once again, persist your changes by committing them to the database.

In [19]:
# Your code here
conn.commit()

In [20]:
conn.close()

## Summary

While there's certainly more to do with setting up and managing this database, you got a taste for creating, populating, and maintaining databases! Feel free to continue fleshing out this exercise for more practice. 